# Iterative Algorithm Design

### Sabryn Bley & Thomas Benzshawel
### CSC 3310 Lab 2
### Fall 2023

1. A paragraph describing a “decision rule” that can be applied to solve to the computational
problem. Provide at least 2 illustrations (test cases) that demonstrate how the rule is applied.
2. High-level pseudocode for an algorithm that uses that rule to solve the computational problem
for any input
3. Provide an explanation and justification for why your algorithm is correct (1-3 paragraphs)
4. Perform an analysis of the worst-case run time using asymptotic notation.
5. A table of your test cases, the answers you expect, and the answers returned by running your
implementation of the algorithm.
6. A table and graph from benchmarking your implementation on problem instances of different
sizes. The benchmarks should support your theoretically-derived run time.
7. Attach all of your source code and test cases in an appendix

### Introduction: TODO

### Set-Up:

### The Code:


### Conclusion: